In [7]:
import face_recognition
import os
import cv2


KNOWN_FACES_DIR = 'known_faces/'
UNKNOWN_FACES_DIR = 'unknown_faces'
TOLERANCE = 0.485
FRAME_THICKNESS = 2
FONT_THICKNESS = 1
MODEL = 'hog'  # 'hog' or 'cnn'

In [2]:
print('Loading known faces...')
known_faces = []
known_names = []

Loading known faces...


In [ ]:
for name in os.listdir(KNOWN_FACES_DIR):
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        encoding = face_recognition.face_encodings(image)[0]
        known_faces.append(encoding)
        known_names.append(name)

In [5]:
def name_to_color(name):
    color = [(ord(c.lower())-97)*8 for c in name[:3]]
    return color

In [8]:
print('Processing unknown faces...')
for filename in os.listdir(UNKNOWN_FACES_DIR):
    print(f'Filename {filename}', end='')
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')
    locations = face_recognition.face_locations(image, model=MODEL)
    encodings = face_recognition.face_encodings(image, locations)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    print(f', found {len(encodings)} face(s)')
    for face_encoding, face_location in zip(encodings, locations):
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        match = None
        if True in results:
            match = known_names[results.index(True)]
            print(f' - {match} from {results}')
            
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])
            color = name_to_color(match)
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)
            
            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2] + 22)
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), FONT_THICKNESS)

    cv2.imshow(filename, image)
    cv2.waitKey(0)
    cv2.destroyWindow(filename)

Processing unknown faces...
Filename 5e6fd443c4854013fa548f54.jpg, found 3 face(s)
 - Donald Trump from [True, True, True, True, True, True, True, False, True, False, True, True, True, False, True, True, True, True, True, True, False, False]
Filename 782633-modi-use.jpg, found 2 face(s)
 - Narendra Modi from [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True]
Filename Amit-Shah-and-Narendra-Modi-696x392.jpg, found 4 face(s)
 - Narendra Modi from [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True]
Filename awaw.jpg, found 2 face(s)
 - Narendra Modi from [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]
Filename d.jpg, found 2 face(s)
 - Donald Trump from [False, True, False, False, Fals